In [26]:
pip install pytorch-pretrained-bert

Note: you may need to restart the kernel to use updated packages.


In [27]:
import logging
import os
import argparse
import random
from tqdm import tqdm, trange
import csv

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

In [28]:
# Logger 
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [29]:
""" Arguments
"""
do_train = True
output_dir = None
max_seq_length = 450      # default
do_lower_case = False 
train_batch_size = 32
eval_batch_size = 8       
learning_rate = 5e-5      
num_train_epochs = 3      # Epochs - less epochs to be used for BERT
warmup_proportion = 0.1   # How to use?
seed = 42                 # Random seed
gradient_accumulation_steps = 1 

# Files : 
data_dir_1 = 'Task1/'
data_dir_2 = 'Task2/'
train_file_1 = 'Task1_train.csv'
dev_file_1 = 'Task1_dev.csv'
train_file_2 = 'Task2_train.csv'
dev_file_2 = 'Task2_dev.csv'


In [30]:
#Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

#Initialise seeds
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)


#Get Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

10/14/2020 01:34:22 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/abhishekmittal/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Now we have to prepare training examples to fine-tune BertForMultipleChoice Model. 

1) First Read the Examples from the csv file.

2) Convert the words into feature vectors by using the BERT Tokenizer

-----------------------Part 1 ------------------------------

In [31]:

def read_examples(input_file):

    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

    examples = [
        {
            "article" : line[0],
            "question" : line[1],

            "options" : [line[2], line[3], line[4], line[5], line[6]],
         
            "label" : int(line[7]) 
         } for line in lines[1:]    # we skip the line with the column names
    ]
    # print(examples[0])
    return examples

# examples = read_examples(input_dir + train_file)

  ----------------------Part 2-------------------------

  Convert tokens to feature vectors 


In [32]:
import sys
import matplotlib.pyplot as plt

class InputFeatures(object):
    def __init__(self, choices_features, label ):
        # We didn't stored tokens in features
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features 

        ]
        self.label = label


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    """Need to check whether truncation really helps, coz it might remove context! :( """

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_examples_to_features(examples, tokenizer, max_seq_length):
    """Loads a data file into a list of `InputBatch`s."""
    
    # Each choice will correspond to a sample on which we run the
    # inference. For a given example, we will create the 5
    # following inputs:
    # - [CLS] article [SEP] question choice_1 [SEP]
    # - [CLS] article [SEP] question choice_2 [SEP]
    # - [CLS] article [SEP] question choice_3 [SEP]
    # - [CLS] article [SEP] question choice_4 [SEP]
    # - [CLS] article [SEP] question choice_5 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 5
    # outputs.
    features = []
    num_tokens_article = []
    count = 0

    for example_index, example in enumerate(examples):
        article_tokens = tokenizer.tokenize(example['article'])
        ques_tokens = tokenizer.tokenize(example['question'])

        # num_tokens_article.append(len(article_tokens) + len(ques_tokens) )   
        # if(num_tokens_article[example_index] > 450):
        #     count += 1

        choices_features = []
        for option_index, option in enumerate(example['options']):
            # We create a copy of the article tokens in order to be
            # able to shrink it according to option_tokens
            article_tokens_option = article_tokens[:]
            q_tokens = ques_tokens + tokenizer.tokenize(option)       # Might also try replacing "@placeholder" with option
            
            # Modifies `context_tokens_choice` and `ending_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            _truncate_seq_pair(article_tokens_option, q_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + article_tokens_option + ["[SEP]"] + q_tokens + ["[SEP]"]
            segment_ids = [0] * (len(article_tokens_option) + 2) + [1] * (len(q_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example['label']
        if example_index < 5:
            logger.info("*** Example ***")
            # logger.info(f"swag_id: {example.swag_id}")
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                logger.info(f"choice: {choice_idx}")
                logger.info(f"tokens: {' '.join(tokens)}")
                logger.info(f"input_ids: {' '.join(map(str, input_ids))}")
                logger.info(f"input_mask: {' '.join(map(str, input_mask))}")
                logger.info(f"segment_ids: {' '.join(map(str, segment_ids))}")
            # if is_training:
                logger.info(f"label: {label}")

        features.append(
            InputFeatures(
                choices_features = choices_features,
                label = label
            )
        )

    # plt.style.use('ggplot')
    # plt.xlabel('Number of words in Article')
    # plt.ylabel('Frequency')
    # plt.title('Task 2 Train Set Graph')
    # plt.hist(num_tokens_article , bins = 100);
    # plt.show();
    # print(len(examples))
    # print(count)

    return features


Now Prepare The MODEL :) :) 


In [33]:
# Prepare model
model = BertForMultipleChoice.from_pretrained('bert-base-uncased',
    num_choices = 5
)

10/14/2020 01:34:27 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/abhishekmittal/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
10/14/2020 01:34:27 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/abhishekmittal/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/wr/st9f550x7q33d_kvjcmrdnn80000gn/T/tmpk_478jjj
10/14/2020 01:34:30 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_

Move the model to GPU and also prints the model : 

12 Encoder Layers

1 Pooler Layer

1 Dropout layer

1 Final Classifier layer


In [34]:
model.to(device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
           

In [35]:
# Get the parameters of the model. 
# All the hidden weights of the model

param_optimizer = list(model.named_parameters())
# print(param_optimizer)

no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]


In [36]:
# Get the train examples from the csv file
train_examples = read_examples(os.path.join(data_dir_1 + train_file_1) )
num_train_steps = int(len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

t_total = num_train_steps

optimizer = BertAdam(optimizer_grouped_parameters,
                         lr = learning_rate,
                         warmup = warmup_proportion,
                         t_total = t_total)

**Let the Training Begin! **

To toggle the mode of model to train, do model.train() and for eval, do model.eval() 

In [37]:
# Define Accuracy check metrics and training utils

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

def classifiction_metric(preds, labels, label_list):
    """ The Metric of classification, input should be numpy format """

    acc = metrics.accuracy_score(labels, preds)

    labels_list = [i for i in range(len(label_list))]

    report = metrics.classification_report(
        labels, preds, labels=labels_list, target_names=label_list, digits=5, output_dict=True)

    return acc, report


In [ ]:
# Training Step : 

global_step = 0
if do_train:
    train_features = convert_examples_to_features(
        train_examples, tokenizer, max_seq_length
        )
    
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", train_batch_size)
    logger.info("  Num steps = %d", num_train_steps)

    # Convert lists into tensors 
    all_input_ids = torch.tensor(select_field(train_features, 'input_ids'), dtype=torch.long)
    all_input_mask = torch.tensor(select_field(train_features, 'input_mask'), dtype=torch.long)
    all_segment_ids = torch.tensor(select_field(train_features, 'segment_ids'), dtype=torch.long)
    all_label = torch.tensor([f.label for f in train_features], dtype=torch.long)

    # Prepare TensorDataset, read here : https://pytorch.org/cppdocs/api/structtorch_1_1data_1_1datasets_1_1_tensor_dataset.html
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)

    train_sampler = RandomSampler(train_data)

    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

    model.train()
    for _ in trange(int(num_train_epochs), desc="Epoch"):
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        # all_preds = np.array([], dtype=int)
        # all_labels = np.array([], dtype=int)

        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            # if n_gpu > 1:
            #     loss = loss.mean() # mean() to average on multi-gpu.
            # if args.fp16 and args.loss_scale != 1.0:
            #     # rescale loss for fp16 training
            #     # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
            #     loss = loss * args.loss_scale
            # if args.gradient_accumulation_steps > 1:
            #     loss = loss / args.gradient_accumulation_steps
            loss.backward()
            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % gradient_accumulation_steps == 0:
                # if args.fp16 or args.optimize_on_cpu:
                #     if args.fp16 and args.loss_scale != 1.0:
                #         # scale down gradients for fp16 training
                #         for param in model.parameters():
                #             if param.grad is not None:
                #                 param.grad.data = param.grad.data / args.loss_scale
                #     is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                #     if is_nan:
                #         logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                #         args.loss_scale = args.loss_scale / 2
                #         model.zero_grad()
                #         continue
                #     optimizer.step()
                #     copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
                # else:
                train_loss = tr_loss / nb_tr_steps
                print("Training loss : ", train_loss)
                # train_acc, train_report = classifiction_metric(all_preds, all_labels, label_list)

                optimizer.step()
                model.zero_grad()
                global_step += 1


10/14/2020 01:34:33 - INFO - __main__ -   *** Example ***
10/14/2020 01:34:33 - INFO - __main__ -   choice: 0
10/14/2020 01:34:33 - INFO - __main__ -   tokens: [CLS] the trains ##pot ##ting author has agreed to become patron of the leith theatre and launch a new fundraising drive . the leith theatre trust took over the lease of the art deco venue from city of edinburgh council last year . it was nearly sold off by the council in 2004 to pay for the rev ##amp of the king ' s theatre before an out ##cr ##y from locals forced a re ##thi ##nk . the fundraising project needs £2 ##50 , 000 . mr welsh , who was born in leith in 1961 , believes it could help edinburgh ' s music scene recover from the loss of a number of city centre venues . it will come back to life temporarily in may when it will be used as the latest pop - up venue for the hidden door festival , which special ##ises in transforming run - down spaces . a major clean - up for that event and a subsequent overhaul , which the £2

10/14/2020 01:34:33 - INFO - __main__ -   input_ids: 101 1996 4499 11008 3436 3166 2038 3530 2000 2468 9161 1997 1996 29011 3004 1998 4888 1037 2047 15524 3298 1012 1996 29011 3004 3404 2165 2058 1996 10084 1997 1996 2396 21933 6891 2013 2103 1997 5928 2473 2197 2095 1012 2009 2001 3053 2853 2125 2011 1996 2473 1999 2432 2000 3477 2005 1996 7065 16613 1997 1996 2332 1005 1055 3004 2077 2019 2041 26775 2100 2013 10575 3140 1037 2128 15222 8950 1012 1996 15524 2622 3791 21853 12376 1010 2199 1012 2720 6124 1010 2040 2001 2141 1999 29011 1999 3777 1010 7164 2009 2071 2393 5928 1005 1055 2189 3496 8980 2013 1996 3279 1997 1037 2193 1997 2103 2803 9356 1012 2009 2097 2272 2067 2000 2166 8184 1999 2089 2043 2009 2097 2022 2109 2004 1996 6745 3769 1011 2039 6891 2005 1996 5023 2341 2782 1010 2029 2569 13087 1999 17903 2448 1011 2091 7258 1012 1037 2350 4550 1011 2039 2005 2008 2724 1998 1037 4745 18181 1010 2029 1996 21853 12376 1010 2199 5574 2003 3517 2000 3477 2005 1010 2097 3499 2009 2000

10/14/2020 01:34:33 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
10/14/2020 01:34:33 - INFO - __main__ -   segm

10/14/2020 01:34:33 - INFO - __main__ -   label: 0
10/14/2020 01:34:33 - INFO - __main__ -   choice: 4
10/14/2020 01:34:33 - INFO - __main__ -   tokens: [CLS] the trains ##pot ##ting author has agreed to become patron of the leith theatre and launch a new fundraising drive . the leith theatre trust took over the lease of the art deco venue from city of edinburgh council last year . it was nearly sold off by the council in 2004 to pay for the rev ##amp of the king ' s theatre before an out ##cr ##y from locals forced a re ##thi ##nk . the fundraising project needs £2 ##50 , 000 . mr welsh , who was born in leith in 1961 , believes it could help edinburgh ' s music scene recover from the loss of a number of city centre venues . it will come back to life temporarily in may when it will be used as the latest pop - up venue for the hidden door festival , which special ##ises in transforming run - down spaces . a major clean - up for that event and a subsequent overhaul , which the £2 ##50 ,

10/14/2020 01:34:33 - INFO - __main__ -   input_ids: 101 1996 4610 1997 1996 2654 1011 3842 7327 2003 2275 2000 4982 2011 1015 1012 1023 1003 2023 2095 1010 1016 1012 1014 1003 1999 2355 1998 2011 1016 1012 1015 1003 1996 2095 2044 1012 1996 2539 1011 3842 9944 15975 2003 3517 2000 4982 2011 1015 1012 1020 1003 2023 2095 1010 4803 2000 1015 1012 1022 1003 2279 2095 1998 1015 1012 1023 1003 1999 2418 1012 1996 7327 2056 3930 2001 2108 3271 2011 5876 2107 2004 2659 3514 7597 1998 1037 15863 9944 3863 3446 1012 2178 5387 6563 2001 1996 2647 2430 2924 1005 1055 4740 2000 23216 1996 9944 15975 4610 2083 2049 5416 1011 9343 4746 1012 2174 1010 1996 3189 2036 7420 2008 2047 7860 2000 3930 2020 6037 1010 2164 1996 4030 7698 1999 2859 1998 8361 3006 18730 1010 1998 20248 18155 26116 13136 1012 2647 5849 5578 9587 9363 7903 2072 2056 7327 18730 2052 1000 2156 3930 4803 1998 12163 1998 10807 15074 2015 4634 1000 1010 2021 1996 3896 2052 2022 17837 2135 3659 2408 2266 2163 1012 2002 3264 1996 3795

...

10/14/2020 01:34:33 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/14/2020 01:34:33 - INFO - __main__ -   lab

10/14/2020 01:34:33 - INFO - __main__ -   input_ids: 101 1996 4104 5439 2252 2056 1037 3522 16053 2179 2008 17927 2308 2372 2001 3569 2011 2058 2093 1011 7728 1997 2216 2040 4194 1012 23110 3790 5444 1999 2089 2025 2000 6449 2308 2372 1998 2439 2049 2157 2000 2754 1996 2330 2528 1012 1000 2057 6160 2023 3247 2011 2548 19817 7828 1998 17614 2049 7784 2005 1996 2252 1010 1000 1996 1054 1004 1037 2056 1012 1000 2256 3579 2003 2200 2172 2006 1996 13741 2705 2330 2021 2057 2064 2085 2298 2830 2000 2116 2062 2307 3219 2012 2548 19817 7828 1999 2086 2000 2272 1012 1000 2548 19817 7828 1010 4350 1999 2148 1037 12541 7430 1010 2003 1996 3677 2005 2023 2095 1005 1055 2330 1010 2029 3216 2013 2403 1011 2459 2251 1012 1999 2254 2009 2056 2009 2052 3319 1997 2049 3287 1011 2069 5779 3343 1012 2252 2952 3235 18178 9654 2056 1024 1000 2057 2031 2056 1037 2193 1997 2335 3728 2008 2009 2003 2590 2005 5439 4184 2000 8339 1996 2554 1999 2029 2057 4839 1998 1996 2715 2088 2008 3504 2000 2149 1012 1000 356

10/14/2020 01:34:33 - INFO - __main__ -   label: 4
10/14/2020 01:34:33 - INFO - __main__ -   choice: 1
10/14/2020 01:34:33 - INFO - __main__ -   tokens: [CLS] former aberdeen and manchester united manager sir alex and the aberdeen - born singer featured in almost 2 , 000 public responses . their achievements will be celebrated at provost sk ##ene ' s house . the building will re ##open following the completion of the maris ##chal square development this summer . the hall of heroes : the 10 were picked from 50 candidates during a three - week vote . aberdeen city council leader jenny lai ##ng said : " the public ' s contribution has been tremendous ##ly valuable and will help ensure the attraction has widespread appeal . " the stories and achievements of our heroes - indeed all the candidates - clearly res ##onate ##d with residents . " percussion ##ist dame evelyn glenn ##ie said : " i am hugely honoured to have been selected . " [SEP] sir alex ferguson and annie lennox are among the @

10/14/2020 01:34:33 - INFO - __main__ -   label: 4
10/14/2020 01:34:33 - INFO - __main__ -   choice: 3
10/14/2020 01:34:33 - INFO - __main__ -   tokens: [CLS] former aberdeen and manchester united manager sir alex and the aberdeen - born singer featured in almost 2 , 000 public responses . their achievements will be celebrated at provost sk ##ene ' s house . the building will re ##open following the completion of the maris ##chal square development this summer . the hall of heroes : the 10 were picked from 50 candidates during a three - week vote . aberdeen city council leader jenny lai ##ng said : " the public ' s contribution has been tremendous ##ly valuable and will help ensure the attraction has widespread appeal . " the stories and achievements of our heroes - indeed all the candidates - clearly res ##onate ##d with residents . " percussion ##ist dame evelyn glenn ##ie said : " i am hugely honoured to have been selected . " [SEP] sir alex ferguson and annie lennox are among the @

10/14/2020 01:34:33 - INFO - __main__ -   label: 4
10/14/2020 01:34:55 - INFO - __main__ -   ***** Running training *****
10/14/2020 01:34:55 - INFO - __main__ -     Num examples = 3227
10/14/2020 01:34:55 - INFO - __main__ -     Batch size = 32
10/14/2020 01:34:55 - INFO - __main__ -     Num steps = 302
Iteration:   0%|          | 0/101 [00:00<?, ?it/s]